In [1]:
# !pip install langchain_community

In [2]:
from langchain_community.llms import Ollama

In [3]:
llm = Ollama(model="llama2")
llm.invoke("The first man on the moon was ...")

' Unterscheidung zwischen "first man" und "first person" :\n\n* The first man on the moon was Neil Armstrong. (This is a statement of fact and refers to the person who was the first human to walk on the moon.)\n* The first person on the moon was Neil Armstrong. (This is a statement of personal opinion or experience and refers to the speaker\'s own perspective or encounter with someone who walked on the moon.)\n\nIn general, "first man" refers to a historical figure or event, while "first person" refers to a personal experience or perspective.'

In [4]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

'Sure! Here are the expenses you provided, along with an appropriate category for each one:\n\n* Spotify AB by Adyen - Entertainment\n* Beta Boulders Ams Amsterdam Nld - Sports\n* Taxi Utrecht - Transportation\n* Ministerie van Justitie en Veiligheid - Government\n* Etos AMSTERDAM NLD - Grocery/Food\n* Bistro Bar Amsterdam - Dining\n\nI hope this helps! Let me know if you have any other questions.'

### Read transaction data

In [5]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("transactions_2022_2023.csv")
df.head()

,Date,Name / Description,Expense/Income,Amount (Rupee)
0,12/30/2023,Raj's Dhaba,Expense,500
1,12/30/2023,Airtel Bill Payment,Expense,300
2,12/30/2023,House Rent,Expense,15000
3,12/30/2023,Sweta's Boutique Purchase,Expense,1200
4,12/29/2023,Freelance Web Design Project,Income,8000


In [6]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

19

In [7]:
unique_transactions[1:10]

array(['Airtel Bill Payment', 'House Rent', "Sweta's Boutique Purchase",
       'Freelance Web Design Project', 'Zomato Food Order',
       'Grocery from D-Mart', 'Tuition Fees Received',
       'Diwali Donation to NGO', 'Salary'], dtype=object)

### Categorise bank transactions with Llama2

In [8]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 19]

In [9]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [10]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['Of course! Here are the appropriate categories for each expense:', '', '1. ISS Catering Services De Meern - Food', '2. Taxi Utrecht - Transportation', '3. Etos AMSTERDAM NLD - Grocery', '4. Bistro Bar Amsterdam - Entertainment']


,Transaction vs category,Transaction,Category
0,Of course! Here are the appropriate categories...,Of course! Here are the appropriate categories...,None
1,,,None
2,1. ISS Catering Services De Meern - Food,1. ISS Catering Services De Meern,Food
3,2. Taxi Utrecht - Transportation,2. Taxi Utrecht,Transportation
4,3. Etos AMSTERDAM NLD - Grocery,3. Etos AMSTERDAM NLD,Grocery
5,4. Bistro Bar Amsterdam - Entertainment,4. Bistro Bar Amsterdam,Entertainment


In [11]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['Sure! Here are the appropriate categories for each of the expenses you provided:', '', "1. Raj's Dhaba - Food", '2. Airtel Bill Payment - Utilities', '3. House Rent - Housing', "4. Sweta's Boutique Purchase - Shopping", '5. Freelance Web Design Project - Work', '6. Zomato Food Order - Food', '7. Grocery from D-Mart - Groceries', '8. Tuition Fees Received - Education', '9. Diwali Donation to NGO - Charity', '10. Salary - Income', '11. Amazon Shopping - Shopping', '12. Reliance Fresh Groceries - Groceries', '13. Income from Rent - Income', '14. Flipkart Shopping - Shopping', '15. Electricity Bill - Utilities', '16. Internet Bill - Utilities', '17. Car EMI - Transportation', '18. Income from Shares - Investments', '19. Restaurant Bill - Dining Out', '', 'Note: Some of these categories may overlap or be similar to each other, but they should be distinct enough to help you track your expenses more accurately.']


In [12]:
categories_df_all

,Transaction vs category,Transaction,Category
0,Sure! Here are the appropriate categories for ...,Sure! Here are the appropriate categories for ...,None
1,,,None
2,1. Raj's Dhaba - Food,1. Raj's Dhaba,Food
3,2. Airtel Bill Payment - Utilities,2. Airtel Bill Payment,Utilities
4,3. House Rent - Housing,3. House Rent,Housing
5,4. Sweta's Boutique Purchase - Shopping,4. Sweta's Boutique Purchase,Shopping
6,5. Freelance Web Design Project - Work,5. Freelance Web Design Project,Work
7,6. Zomato Food Order - Food,6. Zomato Food Order,Food
8,7. Grocery from D-Mart - Groceries,7. Grocery from D-Mart,Groceries
9,8. Tuition Fees Received - Education,8. Tuition Fees Received,Education


In [13]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [14]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array([None, 'Food', 'Utilities', 'Housing', 'Shopping', 'Work',
       'Groceries', 'Education', 'Charity', 'Income', 'Transportation',
       'Investments', 'Dining Out'], dtype=object)

In [15]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [16]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

C:\Users\v-kausha\AppData\Local\Temp\ipykernel_23912\2440992137.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')


,Transaction vs category,Transaction,Category
2,1. Raj's Dhaba - Food,1. Raj's Dhaba,Food and Drinks
3,2. Airtel Bill Payment - Utilities,2. Airtel Bill Payment,Utilities
4,3. House Rent - Housing,3. House Rent,Housing
5,4. Sweta's Boutique Purchase - Shopping,4. Sweta's Boutique Purchase,Shopping
6,5. Freelance Web Design Project - Work,5. Freelance Web Design Project,Work
7,6. Zomato Food Order - Food,6. Zomato Food Order,Food and Drinks
8,7. Grocery from D-Mart - Groceries,7. Grocery from D-Mart,Groceries
9,8. Tuition Fees Received - Education,8. Tuition Fees Received,Education
10,9. Diwali Donation to NGO - Charity,9. Diwali Donation to NGO,Charity
11,10. Salary - Income,10. Salary,Income


In [17]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (Rupee),Transaction vs category,Transaction,Category
0,12/30/2023,Raj's Dhaba,Expense,500,NaN,NaN,NaN
1,12/30/2023,Airtel Bill Payment,Expense,300,NaN,NaN,NaN
2,12/30/2023,House Rent,Expense,15000,NaN,NaN,NaN
3,12/30/2023,Sweta's Boutique Purchase,Expense,1200,NaN,NaN,NaN
4,12/29/2023,Freelance Web Design Project,Income,8000,NaN,NaN,NaN
5,12/29/2023,Zomato Food Order,Expense,450,NaN,NaN,NaN
6,12/23/2023,Grocery from D-Mart,Expense,2000,NaN,NaN,NaN
7,12/22/2023,Tuition Fees Received,Income,6000,NaN,NaN,NaN
8,12/22/2023,Diwali Donation to NGO,Expense,500,NaN,NaN,NaN
9,12/20/2023,Salary,Income,25000,NaN,NaN,NaN


In [18]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)